In [139]:
# @title Copyright & License (click to expand)
# MIT License

# Copyright (c)
# Jiale Shi and Dandan Tang

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Earth Mover's Distance (EMD)-based Kernel Gaussian Processes Regression from Scratch for Mixture Properties Predictions

## References

\[1\] Martin Krasser. [Gaussian Processes from Scratch](https://colab.research.google.com/github/krasserm/bayesian-machine-learning/blob/dev/gaussian-processes/gaussian_processes.ipynb).

\[2\] Kevin P. Murphy. [Machine Learning, A Probabilistic Perspective](https://mitpress.mit.edu/books/machine-learning-0), Chapters 4, 14 and 15.  

\[3\] Christopher M. Bishop. [Pattern Recognition and Machine Learning](http://www.springer.com/de/book/9780387310732), Chapter 6.  

\[4\] Carl Edward Rasmussen and Christopher K. I. Williams. [Gaussian Processes for Machine Learning](http://www.gaussianprocess.org/gpml/).  

In [144]:
import numpy as np
import matplotlib.pyplot as plt
import json
import random
import seaborn as sns
import time

from scipy.optimize import minimize
from scipy.stats import bernoulli
from scipy.special import expit as sigmoid

from sklearn.datasets import make_moons
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

import shutil
from numpy.linalg import inv
from numpy.linalg import cholesky, det
from scipy.linalg import solve_triangular
from scipy.optimize import minimize

import ot
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem.rdFingerprintGenerator import GetRDKitFPGenerator

# EMD Function

$ EMD (P,Q)= \min_{F}\sum_{i=1}^{m}\sum_{j=1}^{n}f_{i,j}d_{i,j}$

$ {\rm subject\ to\ } f_{i,j} \geq 0, {\rm\ for\ any\ } \ 1 \leq i \leq m, 1 \leq j \leq n $

$     \sum_{j=1}^{n}f_{i,j} =w_{pi}, {\rm for\ any} \ 1 \leq i \leq m $

$     \sum_{i=1}^{m}f_{i,j} =w_{qj}, {\rm for\ any} \ 1 \leq j \leq n $

$     \sum_{i=1}^{m}\sum_{j=1}^{n}f_{i,j} = \sum_{i}^{m} w_{pi} = \sum_{j}^{n} w_{qj} = 1 $ 


Molecular mixture/ensemble $P = \{(p_1,w_{p_1}) ,(p_2,w_{p_2}),...,(p_i,w_{p_i}),...,(p_m,w_{p_m})\}$ has $m$ molecular components, where $p_i$ is a molecular component such  and $w_{p_i}>0$ is the weight, related to the average probability (or frequency, $z$) of the molecular component being present in the mixture/ensemble. Similarly, the second mixture/ensemble $Q = {(q_1,w_{q_1}),(q_2,w_{q_2}),...,(q_j,w_{q_j}),...,(q_n,w_{q_n})}$ has $n$ molecular components.  The sums of the weights for $P$ and $Q$ are both normalized and equal to one.
The distance $d_{i,j}$ is calculated by the converting the pairwise similarity score $s_{i,j}$ between $p_{i}$ and $q_{j}$, where $d_{i,j} = 1 - s_{i,j}$.
Transport flow $f_{i,j}$ represents the amount of weight at $p_i$ which is transported to $q_j$. 

The EMD linear optimization code for molecule mixtures refers to [Polymer-Graph-Similarity](https://github.com/olsenlabmit/Polymer-Graph-Similarity/tree/main)


In [43]:
def EMD_Calculation_pot(
    query_smiles_list=None,
    query_smiles_weight_list=None,
    target_smiles_list=None,
    target_smiles_weight_list=None,
    embedding_function="MorganFingerprint",
    similarity_score_function="Tanimoto",
    radius = 2,
    num_bits = 2048,
):
    # obtain the length of query smiles list and target smiles list
    query_smiles_list_length = len(query_smiles_list)

    target_smiles_list_length = len(target_smiles_list)

    query_smiles_weight_list = list(np.ones([query_smiles_list_length]))

    target_smiles_weight_list = list(np.ones([target_smiles_list_length]))


    # transfer SMILES to fingerprints
    if embedding_function == "RDKFingerprint":

        fpgen = GetRDKitFPGenerator(fpSize=num_bits)

        #fps = [fpgen.GetFingerprint(x) for x in ms]

        query_mol_list = [Chem.MolFromSmiles(x) for x in query_smiles_list]
        query_fingerprint_list = [fpgen.GetFingerprint(x)for x in query_mol_list]
        target_mol_list = [Chem.MolFromSmiles(x) for x in target_smiles_list]
        target_fingerprint_list = [fpgen.GetFingerprint(x) for x in target_mol_list]

    elif embedding_function == "MorganFingerprint":
        query_mol_list = [Chem.MolFromSmiles(x) for x in query_smiles_list]
        query_fingerprint_list = [
            AllChem.GetMorganFingerprintAsBitVect(x, radius, nBits=num_bits)
            for x in query_mol_list
        ]
        target_mol_list = [Chem.MolFromSmiles(x) for x in target_smiles_list]
        target_fingerprint_list = [
            AllChem.GetMorganFingerprintAsBitVect(x, radius, nBits=num_bits)
            for x in target_mol_list
        ]

    elif embedding_function == "MACCSkeys":
        query_mol_list = [Chem.MolFromSmiles(x) for x in query_smiles_list]
        query_fingerprint_list = [MACCSkeys.GenMACCSKeys(x) for x in query_mol_list]
        target_mol_list = [Chem.MolFromSmiles(x) for x in target_smiles_list]
        target_fingerprint_list = [MACCSkeys.GenMACCSKeys(x) for x in target_mol_list]

    else:
        print(
            embedding_function
            + " is not included in the current vision."
            + " Please choose an available embedding function:"
        )
        print("MorganFingerprint, RDKFingerprint, MACCSkeys.")
        return False

    # define the required three sets
    C = np.zeros([query_smiles_list_length,target_smiles_list_length ])

    # use similarity function to calculate d_ij
    if similarity_score_function == "Tanimoto":
        for i in range(0, query_smiles_list_length):
            for j in range(0, target_smiles_list_length):
                C[i,j] = 1 - DataStructs.FingerprintSimilarity(
                    query_fingerprint_list[i], target_fingerprint_list[j],
                    metric=DataStructs.TanimotoSimilarity

                )

    elif similarity_score_function == "Dice":
        for i in range(0, query_smiles_list_length):
            for j in range(0, target_smiles_list_length):
                C[i,j] = 1 - DataStructs.FingerprintSimilarity(
                    query_fingerprint_list[i], target_fingerprint_list[j],
                    metric=DataStructs.DiceSimilarity
                )

    elif similarity_score_function == "Cosine":
        for i in range(0, query_smiles_list_length):
            for j in range(0, target_smiles_list_length):
                C[i,j] = 1 - DataStructs.FingerprintSimilarity(
                    query_fingerprint_list[i], target_fingerprint_list[j],
                    metric=DataStructs.CosineSimilarity
                )

    else:
        print(
            similarity_score_function
            + " is not included in the current vision."
            + " Please choose an available similarity function:"
        )
        print("Tanimoto, Dice, or Cosine")
        return

    #print(C)
    query_smiles_weight_array = np.array(query_smiles_weight_list)/sum(np.array(query_smiles_weight_list))
    target_smiles_weight_array = np.array(target_smiles_weight_list)/sum(np.array(target_smiles_weight_list))
    #print(query_smiles_weight_array, target_smiles_weight_array)
    ot_emd = ot.emd(query_smiles_weight_array, target_smiles_weight_array, C)
    #print(ot_emd)

    W = np.sum(ot_emd * C)
    #print(W)
    return W

Here, we will use the squared exponential kernel, also known as Gaussian kernel or RBF kernel.
Here is the default RBF which uses Euclidean distance.
$$
\kappa(\mathbf{x}_P,\mathbf{x}_Q) = \sigma_f^2 \exp\left(-\frac{d(\mathbf{x}_P - \mathbf{x}_Q)^2}{2l^2}\right)
$$

$d(\mathbf{x}_P - \mathbf{x}_Q)^2$ is the Euclidean distance between $\mathbf{x}_P$ and $\mathbf{x}_Q$. The length parameter $l$ controls the smoothness of the function and $\sigma_f$ the vertical variation. For simplicity, we use the same length parameter $l$ for all input dimensions (isotropic kernel).

However, for mixtures, usually it is hard to get the good embedding vector $\mathbf{x}_P$ and $\mathbf{x}_Q$. EMD provides a way to calculate the distance between mixture P and Q from the individual component distance without the needs to get the embedding vector $\mathbf{x}_P$ and $\mathbf{x}_Q$.

$$
\kappa(\mathbf{P},\mathbf{Q}) = \sigma_f^2 \exp\left(-\frac{EMD(\mathbf{P}, \mathbf{Q})^2}{2l^2}\right)
$$

In [64]:
def kernel_EMD(X1, X2, l=1.0, sigma_f=1.0):
    """
    Isotropic squared exponential kernel.

    Args:
        X1: Array of m points (m x d).
        X2: Array of n points (n x d).

    Returns:
        (m x n) matrix.
    """
    m = X1.shape[0]
    n = X2.shape[0]
    sqdist = np.zeros([m,n])
    start_time = time.time()

    for i in np.arange(0,m):
        for j in np.arange(0,n):
            xi = X1[i].item() # dict such as {"COC":0.5, "CC": 0.5}
            xj = X2[j].item() # dict such as {"CCCO": 0.2, "CCS": 0.4, 'CO': 0.4}
            #print(x1, x2)
            emd = EMD_Calculation_pot(query_smiles_list=xi["components"],
                                              query_smiles_weight_list=xi["weights"],
                                              target_smiles_list=xj["components"],
                                              target_smiles_weight_list=xj["weights"],
                                              embedding_function="MorganFingerprint",
                                              similarity_score_function="Tanimoto",
                                              radius = 5,
                                              num_bits = 2048)
            sqdist[i,j] = emd**2
            #print(i,j, sqdist[i,j])
    end_time = time.time()
    print(sqdist.shape, "time_cost:", end_time- start_time, "seconds")
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

In [140]:
# Martin Krasser. [Gaussian Processes from Scratch](https://colab.research.google.com/github/krasserm/bayesian-machine-learning/blob/dev/gaussian-processes/gaussian_processes.ipynb).
def posterior_EMD(X_s, X_train, Y_train, l=1.0, sigma_f=1.0, sigma_y=1e-8):
    """
    Computes the suffifient statistics of the posterior distribution
    from m training data X_train and Y_train and n new inputs X_s.

    Args:
        X_s: New input locations (n x d).
        X_train: Training locations (m x d).
        Y_train: Training targets (m x 1).
        l: Kernel length parameter.
        sigma_f: Kernel vertical variation parameter.
        sigma_y: Noise parameter.

    Returns:
        Posterior mean vector (n x d) and covariance matrix (n x n).
    """
    K = kernel_EMD(X_train, X_train, l, sigma_f) + sigma_y**2 * np.eye(len(X_train))
    K_s = kernel_EMD(X_train, X_s, l, sigma_f)
    K_ss = kernel_EMD(X_s, X_s, l, sigma_f) + 1e-8 * np.eye(len(X_s))
    K_inv = inv(K)

    # Equation (7)
    mu_s = K_s.T.dot(K_inv).dot(Y_train)

    # Equation (8)
    cov_s = K_ss - K_s.T.dot(K_inv).dot(K_s)

    return mu_s, cov_s

In [142]:
#Martin Krasser. [Gaussian Processes from Scratch](https://colab.research.google.com/github/krasserm/bayesian-machine-learning/blob/dev/gaussian-processes/gaussian_processes.ipynb).
def nll_fn_EMD(X_train, Y_train, noise, naive=True):
    """
    Returns a function that computes the negative log marginal
    likelihood for training data X_train and Y_train and given
    noise level.

    Args:
        X_train: training locations (m x d).
        Y_train: training targets (m x 1).
        noise: known noise level of Y_train.
        naive: if True use a naive implementation of Eq. (11), if
               False use a numerically more stable implementation.

    Returns:
        Minimization objective.
    """

    Y_train = Y_train.ravel()

    def nll_naive(theta):
        # Naive implementation of Eq. (11). Works well for the examples
        # in this article but is numerically less stable compared to
        # the implementation in nll_stable below.
        K = kernel_EMD(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))

        return 0.5 * np.log(det(K)) + \
               0.5 * Y_train.dot(inv(K).dot(Y_train)) + \
               0.5 * len(X_train) * np.log(2*np.pi)

    def nll_stable(theta):
        # Numerically more stable implementation of Eq. (11) as described
        # in http://www.gaussianprocess.org/gpml/chapters/RW2.pdf, Section
        # 2.2, Algorithm 2.1.

        K = kernel_EMD(X_train, X_train, l=theta[0], sigma_f=theta[1]) + \
            noise**2 * np.eye(len(X_train))
        L = cholesky(K)

        S1 = solve_triangular(L, Y_train, lower=True)
        S2 = solve_triangular(L.T, S1, lower=False)

        return np.sum(np.log(np.diagonal(L))) + \
               0.5 * Y_train.dot(S2) + \
               0.5 * len(X_train) * np.log(2*np.pi)

    if naive:
        return nll_naive
    else:
        return nll_stable

## Load and Preprocess Dataset

In [143]:
# Replace 'your_file.json' with the path to your actual JSON file
file_path = 'reaction_dh.json'
#file_path = 'reaction_bh.json'

# Open the JSON file for reading
with open(file_path, 'r') as file:
    # Load the file's content as a Python dictionary
    data = json.load(file)
    
X = []
y = []

for i in range(0, len(data["mixture_list"])):
    
    components = data["mixture_list"][i]["components"]
    weights = data["mixture_list"][i]["weights"]
    dict_i = {}
    dict_i['components'] = components
    dict_i["weights"]= weights
    
    X.append(dict_i)
    y.append(data["mixture_list"][i]['figure_of_merit'])
    if None in components:
        print("error")
        
X = np.array(X).reshape(-1,1)
y = np.array(y).reshape(-1,1)       

In [106]:
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle= True, test_size = 0.2, random_state = 2019)

In [115]:
y_train_norm = (y_train-np.mean(y_train))/(np.std(y_train))
y_test_norm = (y_test-np.mean(y_train))/(np.std(y_train))

In [ ]:
res = minimize(nll_fn_EMD(X_train, y_train_norm, noise=0.0, naive=True), [1, 1],
               bounds=((1e-2, None), (1e-2, None)),
               method='L-BFGS-B')#'L-BFGS-B')

In [ ]:
# Store the optimization results in global variables so that we can
# compare it later with the results from other implementations.
l_opt, sigma_f_opt = res.x
print(l_opt, sigma_f_opt)
# Compute posterior mean and covariance with optimized kernel parameters and plot the results
mu_s, cov_s = posterior_EMD(X_test, X_train, y_train_norm, l=l_opt, sigma_f=sigma_f_opt, sigma_y=0.0)

plt.scatter(y_test_norm, mu_s)
plt.show()